In [1]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("PySpark").getOrCreate()

In [2]:
emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

emp = spark.createDataFrame(data=emp_data, schema=emp_schema)

In [3]:
emp.show()

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|
|     

In [4]:
emp.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)



In [5]:
# Create new column using when statement\
from pyspark.sql.functions import when,col,expr
emp_gender_fixed = emp.withColumn("new_gender",when(col("gender") == 'Male', 'M').when(col("gender")=="Female","F").otherwise(None))
emp_gender_fixed_1 = emp.withColumn("new_gender",expr("CASE WHEN gender = 'Male' THEN 'M' WHEN gender == 'Female' THEN 'F' ELSE null END"))

In [6]:
emp_gender_fixed.show()
emp_gender_fixed_1.show()

+-----------+-------------+-------------+---+------+------+----------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|new_gender|
+-----------+-------------+-------------+---+------+------+----------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|         M|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|         F|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|         M|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|         F|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|         M|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|         F|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|         M|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|         F|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|         M|
|        010|   

In [8]:
# Replace in strings

from pyspark.sql.functions import regexp_replace
emp_name_fixed = emp_gender_fixed.withColumn("new_name",regexp_replace(col("name"), "J", "Z"))

In [9]:
emp_name_fixed.show()

+-----------+-------------+-------------+---+------+------+----------+----------+-------------+
|employee_id|department_id|         name|age|gender|salary| hire_date|new_gender|     new_name|
+-----------+-------------+-------------+---+------+------+----------+----------+-------------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|         M|     Zohn Doe|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|         F|   Zane Smith|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|         M|    Bob Brown|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|         F|    Alice Lee|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|         M|    Zack Chan|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|         F|    Zill Wong|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|         M|Zames Zohnson|
|        008|          102|     Kate Kim

In [10]:
# Convert Date
from pyspark.sql.functions import to_date
emp_date_fix = emp_name_fixed.withColumn("hire_date",to_date(col("hire_date"),'yyyy-MM-dd'))


In [12]:
emp_date_fix.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- new_gender: string (nullable = true)
 |-- new_name: string (nullable = true)



In [17]:
# Add date current date, current timestamp, extract year from hire_date
from pyspark.sql.functions import current_timestamp, current_date

columns = {
    "date_now" : current_date(),
    "timestamp_now": current_timestamp()
}

emp_dated = emp_date_fix.withColumns(columns)

In [18]:
emp_dated.show(truncate=False)

+-----------+-------------+-------------+---+------+------+----------+----------+-------------+----------+--------------------------+
|employee_id|department_id|name         |age|gender|salary|hire_date |new_gender|new_name     |date_now  |timestamp_now             |
+-----------+-------------+-------------+---+------+------+----------+----------+-------------+----------+--------------------------+
|001        |101          |John Doe     |30 |Male  |50000 |2015-01-01|M         |Zohn Doe     |2024-09-13|2024-09-13 03:37:56.579132|
|002        |101          |Jane Smith   |25 |Female|45000 |2016-02-15|F         |Zane Smith   |2024-09-13|2024-09-13 03:37:56.579132|
|003        |102          |Bob Brown    |35 |Male  |55000 |2014-05-01|M         |Bob Brown    |2024-09-13|2024-09-13 03:37:56.579132|
|004        |102          |Alice Lee    |28 |Female|48000 |2017-09-30|F         |Alice Lee    |2024-09-13|2024-09-13 03:37:56.579132|
|005        |103          |Jack Chan    |40 |Male  |60000 |201

In [19]:
# Drop null gender records
emp_1 = emp_dated.dropna()

In [20]:
emp_1.show()

+-----------+-------------+-------------+---+------+------+----------+----------+-------------+----------+--------------------+
|employee_id|department_id|         name|age|gender|salary| hire_date|new_gender|     new_name|  date_now|       timestamp_now|
+-----------+-------------+-------------+---+------+------+----------+----------+-------------+----------+--------------------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|         M|     Zohn Doe|2024-09-13|2024-09-13 03:39:...|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|         F|   Zane Smith|2024-09-13|2024-09-13 03:39:...|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|         M|    Bob Brown|2024-09-13|2024-09-13 03:39:...|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|         F|    Alice Lee|2024-09-13|2024-09-13 03:39:...|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|         M|    Zack Chan|2024-09-1

In [26]:
# Fix null records
from pyspark.sql.functions import coalesce, lit
emp_null_df = emp_dated.withColumn("new_gender",coalesce(col("new_gender"),lit("O")))

In [27]:
emp_null_df.show()

+-----------+-------------+-------------+---+------+------+----------+----------+-------------+----------+--------------------+
|employee_id|department_id|         name|age|gender|salary| hire_date|new_gender|     new_name|  date_now|       timestamp_now|
+-----------+-------------+-------------+---+------+------+----------+----------+-------------+----------+--------------------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|         M|     Zohn Doe|2024-09-13|2024-09-13 03:43:...|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|         F|   Zane Smith|2024-09-13|2024-09-13 03:43:...|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|         M|    Bob Brown|2024-09-13|2024-09-13 03:43:...|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|         F|    Alice Lee|2024-09-13|2024-09-13 03:43:...|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|         M|    Zack Chan|2024-09-1

In [28]:
# Drop old columns and rename new columns
emp_final = emp_null_df.drop("name","gender").withColumnRenamed("new_name","name").withColumnRenamed("new_gender","gender")

In [29]:
emp_final.show()

+-----------+-------------+---+------+----------+------+-------------+----------+--------------------+
|employee_id|department_id|age|salary| hire_date|gender|         name|  date_now|       timestamp_now|
+-----------+-------------+---+------+----------+------+-------------+----------+--------------------+
|        001|          101| 30| 50000|2015-01-01|     M|     Zohn Doe|2024-09-13|2024-09-13 03:45:...|
|        002|          101| 25| 45000|2016-02-15|     F|   Zane Smith|2024-09-13|2024-09-13 03:45:...|
|        003|          102| 35| 55000|2014-05-01|     M|    Bob Brown|2024-09-13|2024-09-13 03:45:...|
|        004|          102| 28| 48000|2017-09-30|     F|    Alice Lee|2024-09-13|2024-09-13 03:45:...|
|        005|          103| 40| 60000|2013-04-01|     M|    Zack Chan|2024-09-13|2024-09-13 03:45:...|
|        006|          103| 32| 52000|2018-07-01|     F|    Zill Wong|2024-09-13|2024-09-13 03:45:...|
|        007|          101| 42| 70000|2012-03-15|     M|Zames Zohnson|202